In [1]:
! pip install boto3
! pip install paramiko
! pip install pysftp



  Obtaining dependency information for botocore<1.32.0,>=1.31.17 from https://files.pythonhosted.org/packages/3d/4c/8f97418ad082458a0d8e6d10434227d54edc6166e3197cee6ecf7b0eeec0/botocore-1.31.85-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 68.0 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.106
    Uninstalling botocore-1.20.106:
      Successfully uninstalled botocore-1.20.106
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.6.0 requires botocore<1.31.18,>=1.31.17, but you have botocore 1.31.85 which is incompatible.
  Obtaining dependency information for paramiko from https://files.pythonhosted.org/packages/ad/50/8792484502c8141c20c996b802fefa8435a9c018a2bb440a06b172782118/paramiko-3.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for

In [3]:
import pysftp
import boto3 
import os

In [10]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr  9 16:15:24 2024
 
@author: drangeve
"""
import pandas as pd
from datetime import datetime, timedelta, date
import paramiko

import os

import io
#import csv
 

In [ ]:
header = ['DESTINO',
          'TIPO_PRUEBA',
          'IP_VERSION',
          'MEDIDA',
          'VALOR',
          'UNIDAD_MEDIDA',
          'FECHA',
          'SEMANA',
          'DEVICE_UUID',
          'DEVICE_NAME',
          'CONTEXT']

In [11]:

###############################################################################
bucket = 'pisemestre1'
s3 = boto3.client('s3')

hostname = 'transfer.tigo.com.co'
port = 2222
username = 'inspector11'
password = 'Pongame5enlamateria+++'
 

In [12]:

###############################################################################
 
desired_year = '2024'
desired_month = '06'

In [13]:

 
###############################################################################
 
remote_dir = f'/INSPECTOR01/{desired_year}/{desired_month}'
local_dir = f'raw/{desired_year}_{desired_month}'


In [9]:

################################################################################## 
transport = paramiko.Transport((hostname, port))
transport.connect(username=username, password=password)
 
# Create SFTP session
sftp = paramiko.SFTPClient.from_transport(transport)
 

In [27]:
cont = 0
today = date.today()
# Loop through subdirectories starting with "semana_xx"
for subdir_attr in sftp.listdir_attr(remote_dir):
    subdir = subdir_attr.filename
    last_modified_time = datetime.fromtimestamp(subdir_attr.st_mtime)
    
    if subdir.startswith("semana_") and last_modified_time.date()==today:
        # Download CSV files with prefix directly to local directory
        for filename_attr in sftp.listdir_attr(f"{remote_dir}/{subdir}"):
            filename=filename_attr.filename
            Last_Mod=datetime.fromtimestamp(filename_attr.st_mtime)
    
            if filename.endswith(".csv") and Last_Mod.date()==today:
                local_filename = f"{subdir}+{filename}"
                sftp.get(f"{remote_dir}/{subdir}/{filename}", local_filename)
                
                s3_name = f"{local_dir}/{local_filename}"
                #print(f"Downloaded: {s3_name}")
                
              
                try:
                    s3.upload_file(local_filename,bucket,s3_name)
                    print(f"Downloaded: {s3_name}")
                except:
                    print('Error all cargar')
                    
                    
                    """
                    with sftp.file(remote_file_path, 'r') as remote_file:
                    file_content = remote_file.read().decode('utf-8')

                    # Convertir el contenido del archivo a un DataFrame
                    csv_buffer = io.StringIO(file_content)
                    df = pd.read_csv(csv_buffer)
                    """
               


Downloaded: raw/2024_06/semana_23+a81b9c24985a885afa112df647c67839.csv
Downloaded: raw/2024_06/semana_23+07082e0c5c9d3c7a0209c224c3460c4e.csv
Downloaded: raw/2024_06/semana_23+d23a3aa9c1f7e0789f31b8f6eb018467.csv


In [22]:
 
# Close connection
sftp.close()
transport.close()
 

In [9]:
def trustifyer(df_aux, trusted_path, header,s3):
    
    columnas_utiles = [
                'TIPO_PRUEBA',
                'IP_VERSION',
                'MEDIDA',#'estadistica',
                'DESTINO',#'medicion',
                'VALOR',#'valor',         
                'FECHA',#'fecha',              
                'DEVICE_UUID',#'device',
                'DEVICE_NAME'#'devicename'  
                ]
    
    

    #df_aux= pd.read_csv(data,names=header, low_memory=False, encoding='latin1',
                             usecols = columnas_utiles
                             )
    df_aux = df_aux[
                (df_aux['DESTINO'].isin(['google.com', 'youtube.com'])) &
                (df_aux['TIPO_PRUEBA'] == 'Ping') &
                (df_aux['IP_VERSION'] == 'IPv4') &
                (df_aux['MEDIDA'].isin(['avg', 'packet_loss']))
            ]

        #break    
    df_aux['VALOR'] = df_aux['VALOR'].astype(float)

    print('carga exitosa de semanas')   
                   
    df = df_aux
 
    del df_aux
    #zzz=list(Counter(df['CONTEXT']).keys())
    #del dict_df_raw
    #print('0')#######################################################################################################
    
    df['COL_KEY']= df['IP_VERSION'].fillna('TBD') + '_' + df['DESTINO'].fillna('TBD') + '_' + df['MEDIDA'].fillna('TBD')+  '_' +df['DEVICE_UUID'].fillna('TBD')
    print('creacion columna llave exitoso')
    #print('0')#######################################################################################################

    
    #df = df[columnas_utiles]        
    df['FECHA']=pd.to_datetime(df['FECHA'])
    df['FECHA'] = df['FECHA'].dt.round('30min')        
    print('remuestreo fechas exitoso')
    
    
    #df_cnt = df[(df['DESTINO']=='google.com') & (df['MEDIDA']=='avg') & (df['IP_VERSION'].str.contains('IPv4', case=False))]
    #len(list(set(df_cnt['DEVICE_NAME'])))
###############################################################################
###############################################################################
###############################################################################
###############################################################################
###############################################################################
###############################################################################
    #print('0')#####################################################################################################
    dict_1 = {
        'col_key': 'COL_KEY',
        'estadistica': 'MEDIDA',
        'medicion': 'DESTINO',
        'valor': 'VALOR',
        'fecha': 'FECHA',
        'device': 'DEVICE_UUID',
        'devicename': 'DEVICE_NAME'
    }
    dict_2 = {
        'COL_KEY': 'col_key',
        'MEDIDA': 'estadistica',
        'DESTINO': 'medicion',
        'VALOR': 'valor',
        'FECHA': 'fecha',
        'DEVICE_UUID': 'device',
        'DEVICE_NAME': 'devicename'
    }
###############################################################################
###############################################################################
###############################################################################
###############################################################################
###############################################################################
###############################################################################

#dataframe.rename(columns=column_mapping)
    #print('---')#####################################################################################################
    df = df.rename(columns = dict_2)   
    
    
    
    dict_df = {}
    
    for i in (list(set(list(df['col_key'])))):
        #print('entré al for')
        df_aux=df[df['col_key']==i].reset_index(drop = True)
        #print('1')
        if df_aux['estadistica'].iloc[0] in [ 'upload_speed',
                                     'download_latency',
                                     'HTTP_Time_Total',
                                     'HTTP_Time_Connect',
                                     'HTTP_Time_Start_Transfer',
                                     'mravg',
                                     'HTTP_Time_Redirect',
                                     'HTTP_Time_Name_Lookup',
                                     'HTTP_Time_Handshake',
                                     'HTTP_Time_Pretransfer',
                                     'HTTP_Size_Download',
                                     'download_jitter',
                                     'download_speed',
                                     'avg',
                                     'Query_Time']:
            
            def valor(series):
                return series.iloc[0]
    
            df_aux = df_aux.groupby('fecha').agg({
                'valor': [('mean_valor', 'mean'), ('std_valor', 'std')],
                
                'col_key': valor,
                'medicion': valor,
                'estadistica': valor,
                'device': valor,
                'devicename': valor
            })
            df_aux.reset_index(inplace=True)
            # Flatten the column names
            df_aux.columns = [''.join(col).strip() for col in df_aux.columns.values]
            df_aux.columns = [col.replace('_valor', '').strip() for col in df_aux.columns]
            df_aux.columns = [col.replace('valor', '').strip() for col in df_aux.columns]
            df_aux.rename(columns={'mean': 'valor'}, inplace=True)
            
        #print('2')
        if df_aux['estadistica'].iloc[0] in [ 'DNS_Response',
                                             'HTTP_Code',
                                             'HTTP_SSL_Verify']:
            
            def mode_value(series):
                return series.mode().iloc[0]
    
            def std_value(series):
                return 0
            
            def valor(series):
                return series.iloc[0]
            '''
            result = df_aux.groupby('fecha').agg({
                'valor': [('mean_valor', mode_value)],
                'col_key': valor,
                'medicion': valor,
                'estadistica': valor,
                'device': valor,
                'devicename': valor
            })
            '''
            df_aux.columns = [''.join(col).strip() for col in df_aux.columns.values]
            df_aux.columns = [col.replace('_valor', '').strip() for col in df_aux.columns]
            df_aux.columns = [col.replace('valor', '').strip() if col != 'valor' else col for col in df_aux.columns]
            df_aux['std']=0      
        #print('3')    
        if df_aux['estadistica'].iloc[0] in ['packet_loss']:
            
            def valor(series):
                return series.iloc[0]
    
            df_aux = df_aux.groupby('fecha').agg({
                'valor': [('mean_valor', 'mean')],
                
                'col_key': valor,
                'medicion': valor,
                'estadistica': valor,
                'device': valor,
                'devicename': valor
            })
    
            # Flatten the column names
            df_aux.reset_index(inplace=True)
            df_aux.columns = [''.join(col).strip() for col in df_aux.columns.values]
            df_aux.columns = [col.replace('_valor', '').strip() for col in df_aux.columns]
            #df_aux.columns = [col.replace('valor', '').strip() for col in df_aux.columns]
            df_aux.columns = [col.replace('valor', '').strip() if col != 'valor' else col for col in df_aux.columns]
            df_aux['std']=0
            df_aux.rename(columns={'mean': 'valor'}, inplace=True)
        #print('4')    
        if df_aux['estadistica'].iloc[0] in ['min']:
            
            def min_value(series):
                return series.min()
            
            def valor(series):
                return series.iloc[0]
    
            df_aux = df_aux.groupby('fecha').agg({
                'valor': [('mean_valor', min_value), ('std_valor', std_value)],
                'col_key': valor,
                'medicion': valor,
                'estadistica': valor,
                'device': valor,
                'devicename': valor
            })
    
            # Flatten the column names
            df_aux.columns = [''.join(col).strip() for col in df_aux.columns.values]
            df_aux.columns = [col.replace('_valor', '').strip() for col in df_aux.columns]
            #df_aux.columns = [col.replace('valor', '').strip() for col in df_aux.columns]
            df_aux.columns = [col.replace('valor', '').strip() if col != 'valor' else col for col in df_aux.columns]
            df_aux['std']=0
            df_aux.rename(columns={'mean': 'valor'}, inplace=True)
        #print('5')    
        if df_aux['estadistica'].iloc[0] in ['max']:
            
            def std_value(series):
                return 0
            
            def max_value(series):
                return series.max()
            
            def valor(series):
                return series.iloc[0]
    
            df_aux = df_aux.groupby('fecha').agg({
                'valor': [('mean_valor', max_value)],
                'col_key': valor,
                'medicion': valor,
                'estadistica': valor,
                'device': valor,
                'devicename': valor
            })
    
            # Flatten the column names
            df_aux.columns = [''.join(col).strip() for col in df_aux.columns.values]
            df_aux.columns = [col.replace('_valor', '').strip() for col in df_aux.columns]
            #df_aux.columns = [col.replace('valor', '').strip() for col in df_aux.columns]
            df_aux.columns = [col.replace('valor', '').strip() if col != 'valor' else col for col in df_aux.columns]
            df_aux['std']=0
            df_aux.rename(columns={'mean': 'valor'}, inplace=True)
            
            #break###############################################################################################
        #print('6')    
        dict_df[i] = df_aux
    df_final = pd.concat(dict_df.values(), ignore_index=True)
    csv_buffer = StringIO()

    df_final = df_final.rename(columns = dict_1)  
    
    bucket_name = 'pisemestre1'
    prefix = trusted_path


# Iterar sobre todos los archivos en la carpeta especificada
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    for obj in response.get('Contents', []):
        key = obj['Key']
        if key.endswith('.csv'):
            print(key)
            csv_obj = s3.get_object(Bucket=bucket_name, Key=key)
            body = csv_obj['Body']
            csv_string = body.read().decode('latin1')
            data = StringIO(csv_string)
            Tru= pd.read_csv(data,names=df_final.columns,
                                     usecols =df_final.columns
                                     )
    df_final=pd.concat(Tru,df_final)
    df_final.to_csv(csv_buffer, index=False)
    
    s3.put_object(Bucket='pisemestre1', Key=f'{trusted_path}/trusted.csv', Body=csv_buffer.getvalue())

    

Archivo: semana_23, Última modificación: 2024-06-12
